# 一致预期计算schedule表， 只加载需要计算的ticker的数据

# 批量计算所有 ticker

In [511]:

from datetime import datetime, timedelta
ticker='000002'
pred_date = datetime.today().date()
# pred_date='2019-05-27'
pred_date=datetime(2019,5,27).date()

In [406]:

class MysqlConn(object):
    def __init__(self, config_json_str):
        # self.mysql_config = {"host": "10.22.128.150",  "port": 3317, "db": "bigdata", "user": "talend_load",
        #   "passwd": "s9t5gNThn2vqWM7c" , "charset" : "utf8"}
        # self.mysql_config = {"host": "db-bigdata.wmcloud-qa.com",  "port": 3312, "db": "bigdata", "user": "app_bigdata_ro",
        #   "passwd": "Welcome_20141217"}
        self.mysql_config = json.loads(config_json_str, encoding='utf-8')

    def connect(self):
        mysql_conn = pymysql.connect(**self.mysql_config)
        return mysql_conn

class MssqlConn(object):
    def __init__(self, config_json_str):
        # self.mssql_config = {"server": "sh-datayesdb.wmcloud-dev.com",  "port": 1433, "database": "datayesdb",
        #     "user": "talend_load", "password": "Welcome01"}
        self.mssql_config = json.loads(config_json_str, encoding='utf-8')

    def connect(self):
        mssql_conn = pymssql.connect(**self.mssql_config)
        return mssql_conn

    def __str__(self):
        return json.dumps(self.mssql_config, ensure_ascii=False, encoding='utf-8')



In [407]:
import json
import pymysql
import pymssql
# MySQL
bigdata={"host":"security03-dev.datayes.com","port":3306,"user":"talend_load","passwd":"NCph1G9BQT3DuQj","db":"bigdata","charset":"utf8"}
dateyesdbp={"host":"db-datayesdb-ro.wmcloud.com","port":3313,"user":"app_dataqa_ro","passwd":"Welcome20140820","db":"datayesdbp","charset":"utf8"}
datayesdb={"host":"db-datayesdb-ro.wmcloud.com","port":3313,"database":"datayesdb","user":"app_gaea_ro","password":"EQw6WquhnCKPp8Li","charset":"utf8"}

# researchrpt={"host":"db-researchrpt-ro.wmcloud.com","port":3306,"user":"app_gaea_ro","passwd":"UDVVkvULzakP2048","db":"research_rpt","charset":"utf8"}
# dev
# researchrpt={"host":"10.24.21.34","port":3306,"user":"app_researchrep","passwd":"dG8pqlk5VKOMvDH","db":"research_rpt","charset":"utf8","autocommit":True}
# stg
# researchrpt={"host":"db-researchrpt.wmcloud-stg.com","port":3313,"user":"app_resrpt_rw","passwd":"Wy8tw53XpQfRSukb","db":"research_rpt","charset":"utf8","autocommit":True}
# prd
researchrpt={"host":"db-researchrpt.wmcloud.com","port":3306,"user":"app_researchrep","passwd":"dG8pqlk5VKOMvDH","db":"research_rpt","charset":"utf8","autocommit":True}


# datayesdb_conn = MysqlConn(json.dumps(datayesdb)).connect()
researchrpt_conn = MysqlConn(json.dumps(researchrpt)).connect()

# datayes_conn = MssqlConn(json.dumps(datayes)).connect()


In [408]:
import pandas as pd
import numpy as np

### load data

In [409]:
org_predict_tpas = pd.read_csv("org_predict_tpas.csv", index_col=0, dtype={"ticker_symbol": str})
org_predict_tpas.head()

,report_search_id,ticker_symbol,organ_id,create_date,into_date,time_year,forecast_income,forecast_profit,forecast_eps,org_name,org_remark1
0,100258238,NaN,109,2018-12-31,2019-01-02 00:00:00,2018,3.830000e+09,NaN,NaN,国金证券,1
1,100258238,NaN,109,2018-12-31,2019-01-02 00:00:00,2019,4.540000e+09,NaN,NaN,国金证券,1
2,100258238,NaN,109,2018-12-31,2019-01-02 00:00:00,2020,5.480000e+09,NaN,NaN,国金证券,1
3,100241952,NaN,2570,2019-01-02,2019-01-03 00:00:00,2018,NaN,190000000.0,NaN,天风证券,1
4,100241952,NaN,2570,2019-01-02,2019-01-03 00:00:00,2019,NaN,180000000.0,NaN,天风证券,1


In [410]:
org_predict_tpas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1044114 entries, 0 to 1044113
Data columns (total 11 columns):
report_search_id    1044114 non-null int64
ticker_symbol       1044047 non-null object
organ_id            1044114 non-null int64
create_date         1044114 non-null object
into_date           1044114 non-null object
time_year           1044114 non-null int64
forecast_income     427409 non-null float64
forecast_profit     572442 non-null float64
forecast_eps        1013195 non-null float64
org_name            1044114 non-null object
org_remark1         1044114 non-null int64
dtypes: float64(3), int64(4), object(4)
memory usage: 95.6+ MB


In [411]:
tprice_and_score = pd.read_csv("tprice_and_score.csv", index_col=0, dtype={"ticker_symbol": str})
tprice_and_score = tprice_and_score.dropna(subset=['ticker_symbol'])
tprice_and_score['ticker_symbol'] = tprice_and_score['ticker_symbol'].astype('str')
org_predict_tpas = pd.read_csv("org_predict_tpas.csv", index_col=0, dtype={"ticker_symbol": str})

true = pd.read_csv("true.csv", index_col=0, dtype={"ticker_symbol": str})


In [412]:
#         sql = """
#             select a.id as report_search_id, a.code as ticker_symbol,
#                    a.organ_id, a.create_date, a.text5 as target_price, a.score_id as score
#             from (select id, code, organ_id, create_date, text5, score_id, pause_id, into_date
#                             from cmb_report_research_m 
#                             where pause_id=1  
#                             and length(code)=6
#                             and dt >='{0}' ) a 
#             left join (select id, org_name, org_remark1 from gg_org_list where org_remark1=1) c on a.organ_id=c.id 
#             order by a.code, a.create_date, a.into_date""".format(timestamp)


In [413]:
tprice_and_score.info()
# cmb_report_research_m 共119万数据， 大于16年的数据共51万。

<class 'pandas.core.frame.DataFrame'>
Int64Index: 430323 entries, 42 to 430364
Data columns (total 6 columns):
report_search_id    430323 non-null int64
ticker_symbol       430323 non-null object
organ_id            430320 non-null float64
create_date         430323 non-null object
target_price        77016 non-null float64
score               310131 non-null float64
dtypes: float64(3), int64(1), object(2)
memory usage: 23.0+ MB


In [414]:
tprice_and_score.ticker_symbol.unique()

array(['000001', '000002', '000004', ..., 'A19105', 'A19112', 'DY18028E'],
      dtype=object)

In [415]:
tprice_and_score.head()

,report_search_id,ticker_symbol,organ_id,create_date,target_price,score
42,826638,000001,61.0,2016-09-07,NaN,0.75
43,827140,000001,9.0,2016-09-12,10.4,0.75
44,831604,000001,24.0,2016-10-10,NaN,NaN
45,833186,000001,41.0,2016-10-10,NaN,0.75
46,833098,000001,45.0,2016-10-13,9.4,0.50


In [416]:
tprice_and_score.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 430323 entries, 42 to 430364
Data columns (total 6 columns):
report_search_id    430323 non-null int64
ticker_symbol       430323 non-null object
organ_id            430320 non-null float64
create_date         430323 non-null object
target_price        77016 non-null float64
score               310131 non-null float64
dtypes: float64(3), int64(1), object(2)
memory usage: 23.0+ MB


In [417]:
### primary key: ticker_symbol, organ_id, creaete_date
org_predict.head()

,report_search_id,ticker_symbol,organ_id,create_date,into_date,time_year,forecast_income,forecast_profit,forecast_eps,org_name,org_remark1
0,1169249,000001,9,2016-09-12,2016-09-12 15:55:00,2016.0,NaN,NaN,1.36,海通证券,1.0
1,1169249,000001,9,2016-09-12,2016-09-12 15:55:00,2017.0,NaN,NaN,1.43,海通证券,1.0
2,1173133,000001,45,2016-10-13,2016-10-14 15:25:34,2016.0,1.096580e+11,2.379500e+10,1.39,UBS,1.0
3,1173133,000001,45,2016-10-13,2016-10-14 15:25:34,2017.0,1.194020e+11,2.447400e+10,1.43,UBS,1.0
4,1173133,000001,45,2016-10-13,2016-10-14 15:25:34,2018.0,1.303380e+11,2.538500e+10,1.48,UBS,1.0


In [418]:
true.head(1)

,ticker_symbol,act_pubtime,publish_date,end_date_rep,time_year,income_true,profit_true,eps_true
0,000001,2001-04-13 18:00:00,2001-04-14,2000-12-31,2000,2.789163e+09,506551785.0,NaN


In [419]:
tickers=pd.read_csv("tickers.csv", index_col=0, dtype={"ticker_symbol":str})

In [420]:
tickers.head(1)

,ticker_symbol,list_date,delist_date,SEC_SHORT_NAME
0,000001,1991-04-03,NaN,平安银行


In [16]:
all_year_error_dict={}

# 目标价 target_price

In [435]:

def get_period_data(data, start_date, end_date):
    """
    获取某一时间段内的数据
    """
    # if 'create_date' not in data.columns:
    #     return pd.DataFrame()
    data['create_date'] = pd.to_datetime(data['create_date'])
    start_date = datetime.strptime(start_date, '%Y-%m-%d')
    end_date = datetime.strptime(end_date, '%Y-%m-%d')
    if start_date is None and end_date is None:
        period_data = data.reset_index().drop('index', axis=1)
    elif start_date is not None and end_date is None:
        period_data = data[data['create_date'] >= start_date].reset_index().drop('index', axis=1)
    elif start_date is None and end_date is not None:
        period_data = data[data['create_date'] <= end_date].reset_index().drop('index', axis=1)
    else:
        period_data = data[
            (data['create_date'] >= start_date) & (data['create_date'] <= end_date)].reset_index().drop('index',
                                                                                                        axis=1)
    return period_data


### tprice_and_score

In [512]:
 str(pred_date)

'2019-05-27'

In [513]:
# 筛选90天数据
price_df = get_period_data(tprice_and_score, str(pred_date - timedelta(90)), str(pred_date))

price_df = price_df.dropna(subset=["target_price"]).sort_values(['organ_id', 'create_date'])
# .drop_duplicates(
#     'organ_id', keep='last')

In [514]:
# price_df.duplicated(['ticker_symbol', 'organ_id'], keep='last')

In [515]:
price_df = price_df[~price_df.duplicated(['ticker_symbol', 'organ_id'], keep='last')]

In [516]:
price_df[price_df.report_search_id==2108407]

,report_search_id,ticker_symbol,organ_id,create_date,target_price,score
72,2108407,000002,2.0,2019-04-16,43.01,1.0


In [517]:
price_df.head(1)

,report_search_id,ticker_symbol,organ_id,create_date,target_price,score
2238,2097273,002007,2.0,2019-02-26,45.0,1.0


In [518]:
price_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3266 entries, 2238 to 295
Data columns (total 6 columns):
report_search_id    3266 non-null int64
ticker_symbol       3266 non-null object
organ_id            3266 non-null float64
create_date         3266 non-null datetime64[ns]
target_price        3266 non-null float64
score               3265 non-null float64
dtypes: datetime64[ns](1), float64(3), int64(1), object(1)
memory usage: 178.6+ KB


In [519]:
# price_df
# 按日期筛选后的记录没剩下多少 。 ？？？

In [520]:
# 针对每个ticker， organ_id去重 TODO
# price_df = price_df.drop_duplicates('organ_id', keep='last')
# tmp

In [521]:
tmp = price_df[price_df.ticker_symbol==ticker].reset_index()
# tmp.duplicated(['ticker_symbol', 'organ_id'], keep='last') # keep = 'last' means false,  to get rows are duplicated

In [522]:
tmp

,index,report_search_id,ticker_symbol,organ_id,create_date,target_price,score
0,72,2108407,000002,2.0,2019-04-16,43.01,1.00
1,71,2108070,000002,5.0,2019-04-16,39.91,1.00
2,62,2109673,000002,9.0,2019-03-30,33.21,0.75
3,54,2102450,000002,12.0,2019-03-26,33.21,1.00
4,52,2103228,000002,13.0,2019-03-26,37.05,1.00
5,68,2104598,000002,20.0,2019-04-04,35.00,1.00
6,53,2102345,000002,30.0,2019-03-26,34.38,1.00
7,70,2105829,000002,61.0,2019-04-08,39.60,1.00
8,56,2102915,000002,77.0,2019-03-26,32.58,1.00
9,57,2102347,000002,119.0,2019-03-27,41.13,0.75


### org_predict_tpas

In [523]:
predict_df = get_period_data(org_predict_tpas, str(pred_date - timedelta(90)), str(pred_date))

In [524]:
predict_df.head(1)

,report_search_id,ticker_symbol,organ_id,create_date,into_date,time_year,forecast_income,forecast_profit,forecast_eps,org_name,org_remark1
0,2099592,000001,3,2019-03-03,2019-03-03 00:00:00,2018,1.167560e+11,2.481600e+10,1.45,中金,1


In [525]:
predict_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54224 entries, 0 to 54223
Data columns (total 11 columns):
report_search_id    54224 non-null int64
ticker_symbol       54224 non-null object
organ_id            54224 non-null int64
create_date         54224 non-null datetime64[ns]
into_date           54224 non-null object
time_year           54224 non-null int64
forecast_income     52537 non-null float64
forecast_profit     52905 non-null float64
forecast_eps        53657 non-null float64
org_name            54224 non-null object
org_remark1         54224 non-null int64
dtypes: datetime64[ns](1), float64(3), int64(4), object(3)
memory usage: 4.6+ MB


In [526]:
predict_df = predict_df.dropna(subset=['time_year'])
predict_df['time_year'] = predict_df['time_year'].astype(int)


In [527]:
predict_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54224 entries, 0 to 54223
Data columns (total 11 columns):
report_search_id    54224 non-null int64
ticker_symbol       54224 non-null object
organ_id            54224 non-null int64
create_date         54224 non-null datetime64[ns]
into_date           54224 non-null object
time_year           54224 non-null int32
forecast_income     52537 non-null float64
forecast_profit     52905 non-null float64
forecast_eps        53657 non-null float64
org_name            54224 non-null object
org_remark1         54224 non-null int64
dtypes: datetime64[ns](1), float64(3), int32(1), int64(3), object(3)
memory usage: 4.8+ MB


In [528]:
predict_df.create_date.max()


Timestamp('2019-05-27 00:00:00')

In [529]:
predict_df.create_date.min()


Timestamp('2019-02-26 00:00:00')

In [530]:
predict_df[predict_df.ticker_symbol==ticker].reset_index().info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158 entries, 0 to 157
Data columns (total 12 columns):
index               158 non-null int64
report_search_id    158 non-null int64
ticker_symbol       158 non-null object
organ_id            158 non-null int64
create_date         158 non-null datetime64[ns]
into_date           158 non-null object
time_year           158 non-null int32
forecast_income     149 non-null float64
forecast_profit     146 non-null float64
forecast_eps        157 non-null float64
org_name            158 non-null object
org_remark1         158 non-null int64
dtypes: datetime64[ns](1), float64(3), int32(1), int64(4), object(3)
memory usage: 14.3+ KB


### 取time_year 

In [557]:
# drop duplicate for each ticker, time_year
year_list = predict_df[~predict_df.duplicated(['ticker_symbol','time_year'], keep='last')]
# year_list = predict_df.drop_duplicates('time_year', keep='last').sort_values(by='time_year').reset_index().drop(
#     'index', axis=1)

In [558]:
year_list[year_list.ticker_symbol==ticker]

,report_search_id,ticker_symbol,organ_id,create_date,into_date,time_year,forecast_income,forecast_profit,forecast_eps,org_name,org_remark1
197,2114284,000002,2570,2019-04-30,2019-04-30 00:00:00,2018,NaN,NaN,NaN,天风证券,1
240,2119019,000002,118,2019-05-26,2019-05-26 00:00:00,2019,3.592512e+11,3.910910e+10,3.54,中泰证券,1
241,2119019,000002,118,2019-05-26,2019-05-26 00:00:00,2020,4.425022e+11,4.539274e+10,4.11,中泰证券,1
242,2119019,000002,118,2019-05-26,2019-05-26 00:00:00,2021,5.496804e+11,5.287525e+10,4.79,中泰证券,1


In [559]:
# year_list = year_list[year_list.apply(lambda x: check_already_publish_report(x, new_pubdate, pred_date), axis=1)]


In [560]:
year_list[year_list.ticker_symbol==ticker]

,report_search_id,ticker_symbol,organ_id,create_date,into_date,time_year,forecast_income,forecast_profit,forecast_eps,org_name,org_remark1
197,2114284,000002,2570,2019-04-30,2019-04-30 00:00:00,2018,NaN,NaN,NaN,天风证券,1
240,2119019,000002,118,2019-05-26,2019-05-26 00:00:00,2019,3.592512e+11,3.910910e+10,3.54,中泰证券,1
241,2119019,000002,118,2019-05-26,2019-05-26 00:00:00,2020,4.425022e+11,4.539274e+10,4.11,中泰证券,1
242,2119019,000002,118,2019-05-26,2019-05-26 00:00:00,2021,5.496804e+11,5.287525e+10,4.79,中泰证券,1


### true

In [26]:
true.head(1)

,ticker_symbol,act_pubtime,publish_date,end_date_rep,time_year,income_true,profit_true,eps_true
0,000001,2001-04-13 18:00:00,2001-04-14,2000-12-31,2000,2.789163e+09,506551785.0,NaN


In [27]:
# true[['ticker_symbol', 'act_pubtime', 'time_year']].reset_index().drop('index', axis=1)

In [561]:
new_pubdate = true[['ticker_symbol', 'act_pubtime', 'time_year']].reset_index().drop('index', axis=1)
new_pubdate.head(1)



,ticker_symbol,act_pubtime,time_year
0,000001,2001-04-13 18:00:00,2000


In [562]:
new_pubdate[new_pubdate.ticker_symbol=='000002'].reset_index()

,index,ticker_symbol,act_pubtime,time_year
0,19,000002,2004-03-08 18:00:00,2003
1,20,000002,2005-03-21 18:00:00,2004
2,21,000002,2006-03-20 18:00:00,2005
3,22,000002,2007-03-19 18:00:00,2006
4,23,000002,2008-03-20 18:00:00,2007
5,24,000002,2009-03-09 18:00:00,2008
6,25,000002,2010-03-01 18:00:00,2009
7,26,000002,2011-03-07 18:00:00,2010
8,27,000002,2012-03-12 18:00:00,2011
9,28,000002,2013-02-27 18:00:00,2012


In [563]:
# 
year_list_filter = year_list.merge(new_pubdate[['ticker_symbol','time_year','act_pubtime']], on=['ticker_symbol','time_year'], how='left')

In [567]:
year_list_filter[year_list_filter.ticker_symbol==ticker]

,report_search_id,ticker_symbol,organ_id,create_date,into_date,time_year,forecast_income,forecast_profit,forecast_eps,org_name,org_remark1
5,2119019,000002,118,2019-05-26,2019-05-26 00:00:00,2019,3.592512e+11,3.910910e+10,3.54,中泰证券,1
6,2119019,000002,118,2019-05-26,2019-05-26 00:00:00,2020,4.425022e+11,4.539274e+10,4.11,中泰证券,1
7,2119019,000002,118,2019-05-26,2019-05-26 00:00:00,2021,5.496804e+11,5.287525e+10,4.79,中泰证券,1


In [566]:
# 考查time year, 将实际发布日期小于当前预测日期的记录从year list里移除
year_list_filter = year_list_filter[~(year_list_filter.act_pubtime < str(pred_date))]
year_list_filter = year_list_filter.drop('act_pubtime',axis=1)

In [548]:

# def check_already_publish_report(rows, new_pubdate):
#     """
#     个股已经发布上一年的年报，但需要过滤
#     """
#     # 类型转换为int, 方便比较
#     # new_pubdate['time_year'].astype(int)
#     if len(rows) == 0:
#         return True
#     time_year = int(rows['time_year'])
#     _new_pubdate = new_pubdate[new_pubdate['time_year'].astype(int) == time_year]
#     if len(_new_pubdate) > 0:
#         pass
#     # 之前用的date类型做比较
#     # 为了节省时间，从本地保存的数据文件中读取，有个问题是date对象会读成object, 将其都转为str再比较
#     if len(_new_pubdate) > 0 and str(_new_pubdate['act_pubtime'].iloc[0]) < str(self.pred_date):
#         return False
#     return True

In [69]:
# year_list = year_list[year_list.apply(lambda x: check_already_publish_report(x, new_pubdate, pred_date), axis=1)]


In [70]:
# year_list.head(1)

,report_search_id,ticker_symbol,organ_id,create_date,into_date,time_year,forecast_income,forecast_profit,forecast_eps,org_name,org_remark1
5,2118687,A19057,38,2019-04-13,2019-04-13 00:00:00,2019,4.158440e+09,338770000.0,NaN,方正证券,1


In [568]:
# TODO done
def get_year_error(predict_org):
#     year = predict_org.time_year.unique()
#     true = true[true['time_year'] == year].reset_index().drop('index', axis=1)
    # 这一年度有哪些机构作了预测
    predict_org = predict_org.sort_values(by='create_date').drop_duplicates(
        subset=['organ_id', 'create_date'],
        keep='last').reset_index().drop(
        'index',
        axis=1)
    predict_org['income_true'] = predict_org.income_true.astype(float)
    predict_org['eps_true'] = predict_org.eps_true.astype(float)
    predict_org['forecast_income'] = predict_org.forecast_income.astype(float)
    predict_org['forecast_profit'] = predict_org.forecast_profit.astype(float)
    predict_org['forecast_eps'] = predict_org.forecast_eps.astype(float)
    if len(predict_org) > 0:
        predict_org['income_err'] = ((predict_org.ix[0, 'income_true'] - predict_org['forecast_income']) / predict_org.ix[
            0, 'income_true']) ** 2
        predict_org['profit_err'] = ((predict_org.ix[0, 'profit_true'] - predict_org['forecast_profit']) / predict_org.ix[
            0, 'profit_true']) ** 2
        predict_org['eps_err'] = ((predict_org.ix[0, 'eps_true'] - predict_org['forecast_eps']) / predict_org.ix[
            0, 'eps_true']) ** 2
#         predict_org['time_year'] = year
        return predict_org[['organ_id', 'income_err', 'profit_err', 'eps_err']]


In [569]:
# TODO 这个步骤似乎可以放到初始化数据里了，一次性处理完成
# 先合并 org_predict , true
org_true = org_predict_tpas[["ticker_symbol","organ_id","create_date",'time_year','forecast_income','forecast_profit','forecast_eps']].merge(true[["ticker_symbol",'time_year','income_true', 'profit_true','eps_true']], on=["ticker_symbol",'time_year'])

In [47]:
org_true.head(10)

,ticker_symbol,organ_id,create_date,time_year,forecast_income,forecast_profit,forecast_eps,income_true,profit_true,eps_true
0,000001,61,2016-09-07,2016,NaN,2.334700e+10,1.359723,1.077150e+11,2.259900e+10,1.32
1,000001,9,2016-09-12,2016,NaN,NaN,1.360000,1.077150e+11,2.259900e+10,1.32
2,000001,24,2016-10-10,2016,NaN,2.386687e+10,1.390000,1.077150e+11,2.259900e+10,1.32
3,000001,41,2016-10-10,2016,NaN,2.403858e+10,1.400000,1.077150e+11,2.259900e+10,1.32
4,000001,45,2016-10-13,2016,1.096580e+11,2.379500e+10,1.390000,1.077150e+11,2.259900e+10,1.32
5,000001,109,2016-10-20,2016,1.128610e+11,2.329600e+10,1.360000,1.077150e+11,2.259900e+10,1.32
6,000001,7,2016-10-20,2016,NaN,2.310000e+10,1.340000,1.077150e+11,2.259900e+10,1.32
7,000001,3,2016-10-20,2016,1.125680e+11,2.351900e+10,1.370000,1.077150e+11,2.259900e+10,1.32
8,000001,26,2016-10-20,2016,1.099550e+11,2.303000e+10,1.340000,1.077150e+11,2.259900e+10,1.32
9,000001,13,2016-10-21,2016,1.047750e+11,2.345400e+10,1.370000,1.077150e+11,2.259900e+10,1.32


In [48]:
org_true.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 599185 entries, 0 to 599184
Data columns (total 10 columns):
ticker_symbol      599185 non-null object
organ_id           599185 non-null int64
create_date        599185 non-null datetime64[ns]
time_year          599185 non-null int64
forecast_income    217813 non-null float64
forecast_profit    334491 non-null float64
forecast_eps       583951 non-null float64
income_true        599185 non-null float64
profit_true        599185 non-null float64
eps_true           598938 non-null float64
dtypes: datetime64[ns](1), float64(6), int64(2), object(1)
memory usage: 50.3+ MB


In [602]:
# group by ticker , time_year 
org_error_df = org_true.groupby(["ticker_symbol", "time_year"]).apply(get_year_error)

c:\python27\lib\site-packages\ipykernel_launcher.py:17: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [603]:
org_error_df = org_error_df.reset_index()

In [618]:
# year_error_df = year_error_df.drop('level_2', axis=1)
org_error_df[org_error_df.ticker_symbol==ticker].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1338 entries, 695 to 2032
Data columns (total 7 columns):
ticker_symbol    1338 non-null object
time_year        1338 non-null int64
level_2          1338 non-null int64
organ_id         1338 non-null int64
income_err       545 non-null float64
profit_err       662 non-null float64
eps_err          1320 non-null float64
dtypes: float64(3), int64(3), object(1)
memory usage: 83.6+ KB


In [585]:
# year_error_df.info()

In [41]:
# year_error_df.head(100)

In [42]:
# org_true.groupby(["ticker_symbol", "time_year"]).count()

In [43]:
# org_true.groupby(["ticker_symbol"]).count()

In [575]:
# org_predict_tpas.head(1)

In [574]:
# true.head(10)

In [46]:
# def get_all_year_error(prediction, true):
#     years = list(set(true['time_year'].tolist()) & set(prediction['time_year'].tolist()))
#     all_year_error = pd.DataFrame(columns=['organ_id', 'income_err', 'profit_err', 'eps_err', 'time_year'])
#     for iyear in range(len(years)):
#         all_year_error = all_year_error.append(
#             get_one_year_error(prediction[prediction['time_year'] == years[iyear]].copy(deep=True), true,
#                                     years[iyear]), ignore_index=True)
#     return all_year_error


In [47]:
# # 挑出预测不准的？？？
# # if all_year_error_dict.get(ticker) is None:
# all_year_error_dict[ticker] = get_all_year_error(org_predict_tpas, true)
# all_year_error = all_year_error_dict.get(ticker)
# # else:zzz
# #     all_year_error = all_year_error_dict.get(ticker)


In [48]:
# org_predict_tpas.groupby(["ticker_symbol", "time_year"]).apply(get_one_year_error)

In [49]:
# 根据 true中的 time_year 对 org predict 中的行做筛选， 测试单独一个ticker

In [50]:
# ticker_index=org_predict_tpas[org_predict_tpas.ticker_symbol=='603777'].index

In [51]:
# ticker_index

In [52]:
# org_predict_tpas.head(1)

In [53]:
# orgpre=org_predict_tpas[org_predict_tpas.index.isin(ticker_index)]

In [54]:
# orgpre

In [55]:
# df = get_all_year_error(orgpre, true)

In [56]:
# df.info()

In [57]:
# 挑出预测不准的？？？
# if all_year_error_dict.get(ticker) is None:
#     all_year_error_dict[ticker] = get_all_year_error(org_predict_tpas, true)
#     all_year_error = all_year_error_dict.get(ticker)
# else:zzz
#     all_year_error = all_year_error_dict.get(ticker)


In [58]:
# true_value[['ticker_symbol', 'act_pubtime', 'time_year']].reset_index().drop('index', axis=1)

In [59]:
# TODO 
# true['time_year'].tolist()

In [60]:
# prediction['time_year'].tolist()

In [61]:
# merge df, dfmergeb

In [62]:
price_df.head(1)

,report_search_id,ticker_symbol,organ_id,create_date,target_price,score
16381,2110693,603816,2.0,2019-04-25,64.8,1.0


In [63]:
predict_df.head(1)

,report_search_id,ticker_symbol,organ_id,create_date,into_date,time_year,forecast_income,forecast_profit,forecast_eps,org_name,org_remark1
0,2110748,000001,61,2019-04-23,2019-04-24 00:00:00,2019,1.281590e+11,2.648200e+10,1.49,广发证券,1


In [576]:
# 从 org predict 中去掉price_and_score中没有的数据

dfmerge = predict_df.merge(
    price_df[['report_search_id', 'ticker_symbol', 'organ_id', 'create_date', 'target_price']],
    on=['report_search_id', 'ticker_symbol', 'organ_id', 'create_date'], how='right')

In [578]:
dfmerge[dfmerge.ticker_symbol==ticker].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36 entries, 20 to 55
Data columns (total 12 columns):
report_search_id    36 non-null int64
ticker_symbol       36 non-null object
organ_id            36 non-null float64
create_date         36 non-null datetime64[ns]
into_date           36 non-null object
time_year           36 non-null float64
forecast_income     36 non-null float64
forecast_profit     36 non-null float64
forecast_eps        36 non-null float64
org_name            36 non-null object
org_remark1         36 non-null float64
target_price        36 non-null float64
dtypes: datetime64[ns](1), float64(7), int64(1), object(3)
memory usage: 3.7+ KB


In [77]:
dfmerge.head()

,report_search_id,ticker_symbol,organ_id,create_date,into_date,time_year,forecast_income,forecast_profit,forecast_eps,org_name,org_remark1,target_price
0,2105068,000157,15467.0,2019-04-03,2019-04-03 00:00:00,2019.0,3.303200e+10,2.640000e+09,0.34,申万宏源证券,1.0,5.1
1,2105068,000157,15467.0,2019-04-03,2019-04-03 00:00:00,2020.0,3.261300e+10,3.007000e+09,0.39,申万宏源证券,1.0,5.1
2,2105068,000157,15467.0,2019-04-03,2019-04-03 00:00:00,2021.0,2.920900e+10,2.836000e+09,0.36,申万宏源证券,1.0,5.1
3,2110213,000401,77.0,2019-04-24,2019-04-24 00:00:00,2019.0,3.706300e+10,2.295000e+09,1.70,华泰证券,1.0,18.7
4,2110213,000401,77.0,2019-04-24,2019-04-24 00:00:00,2020.0,3.881800e+10,2.647000e+09,1.96,华泰证券,1.0,18.7


In [78]:
dfmerge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 98 entries, 0 to 97
Data columns (total 12 columns):
report_search_id    98 non-null int64
ticker_symbol       98 non-null object
organ_id            98 non-null float64
create_date         98 non-null datetime64[ns]
into_date           94 non-null object
time_year           94 non-null float64
forecast_income     94 non-null float64
forecast_profit     94 non-null float64
forecast_eps        94 non-null float64
org_name            94 non-null object
org_remark1         94 non-null float64
target_price        98 non-null float64
dtypes: datetime64[ns](1), float64(7), int64(1), object(3)
memory usage: 10.0+ KB


In [56]:
# con_expec = pd.DataFrame(
#             columns=['code', 'pred_date', 'con_' + 'target_price', 'con_type_' + 'target_price'],
#             index=range(1))

### if len(dfmerge['time_year'].dropna()) > 0:

In [579]:
ticker

'000002'

In [580]:
year_error_df.head()

,ticker_symbol,time_year,level_2,organ_id,income_err,profit_err,eps_err
0,000001,2015,0,20,NaN,NaN,0.000370
1,000001,2016,0,61,NaN,0.001096,0.000906
2,000001,2016,1,9,NaN,NaN,0.000918
3,000001,2016,2,24,NaN,0.003148,0.002812
4,000001,2016,3,41,NaN,0.004058,0.003673


In [63]:
# year_error_df = year_error_df.reset_index()

In [59]:
# 选出指定ticker数据
# all_year_error=year_error_df.set_index("ticker_symbol").loc[ticker]

In [60]:
# all_year_error.info()

In [61]:
# all_year_error.head()

In [588]:
year_list.head()

,report_search_id,ticker_symbol,organ_id,create_date,into_date,time_year,forecast_income,forecast_profit,forecast_eps,org_name,org_remark1
61,2110425,000001,20,2019-04-24,2019-04-24 00:00:00,2018,1.167160e+11,2.481800e+10,1.39,中信建投,1
82,2113814,000001,217,2019-05-08,2019-05-08 00:00:00,2019,1.322580e+11,2.759800e+10,1.61,安信证券,1
83,2113814,000001,217,2019-05-08,2019-05-08 00:00:00,2020,1.504000e+11,3.201700e+10,1.86,安信证券,1
84,2113814,000001,217,2019-05-08,2019-05-08 00:00:00,2021,1.701450e+11,3.764200e+10,2.19,安信证券,1
197,2114284,000002,2570,2019-04-30,2019-04-30 00:00:00,2018,NaN,NaN,NaN,天风证券,1


In [589]:
year_list['time_year'].min()

2010

In [587]:
org_error_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 584118 entries, 0 to 584117
Data columns (total 7 columns):
ticker_symbol    584118 non-null object
time_year        584118 non-null int64
level_2          584118 non-null int64
organ_id         584118 non-null int64
income_err       214698 non-null float64
profit_err       330622 non-null float64
eps_err          568996 non-null float64
dtypes: float64(3), int64(3), object(1)
memory usage: 31.2+ MB


In [643]:
year_list_filter[year_list_filter.ticker_symbol==ticker].time_year.min()

2019

In [654]:
# 筛选3年的数据
# 分ticker进行
df = year_list_filter.groupby('ticker_symbol').apply(lambda x: x['time_year'].min()).to_frame('year_min').reset_index()
year_error_df = org_error_df.merge(df, on='ticker_symbol',how='left')
# pred_error = year_error_df[year_error_df['time_year'] >= (year_list['time_year'].min() - 3)]
# pred_error = pred_error[pred_error['time_year'] < (year_list['time_year'].min())]
# pred_error = pred_error[pred_error['time_year'] < year_list['time_year'].min()]


In [655]:
year_error_df.head()

,ticker_symbol,time_year,level_2,organ_id,income_err,profit_err,eps_err,year_min
0,000001,2015,0,20,NaN,NaN,0.000370,2019.0
1,000001,2016,0,61,NaN,0.001096,0.000906,2019.0
2,000001,2016,1,9,NaN,NaN,0.000918,2019.0
3,000001,2016,2,24,NaN,0.003148,0.002812,2019.0
4,000001,2016,3,41,NaN,0.004058,0.003673,2019.0


In [656]:
year_error_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 584118 entries, 0 to 584117
Data columns (total 8 columns):
ticker_symbol    584118 non-null object
time_year        584118 non-null int64
level_2          584118 non-null int64
organ_id         584118 non-null int64
income_err       214698 non-null float64
profit_err       330622 non-null float64
eps_err          568996 non-null float64
year_min         503134 non-null float64
dtypes: float64(4), int64(3), object(1)
memory usage: 40.1+ MB


In [657]:
df=year_error_df[year_error_df.ticker_symbol==ticker]

In [658]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1338 entries, 695 to 2032
Data columns (total 8 columns):
ticker_symbol    1338 non-null object
time_year        1338 non-null int64
level_2          1338 non-null int64
organ_id         1338 non-null int64
income_err       545 non-null float64
profit_err       662 non-null float64
eps_err          1320 non-null float64
year_min         1338 non-null float64
dtypes: float64(4), int64(3), object(1)
memory usage: 94.1+ KB


In [659]:
# df

In [660]:
# year_error_df[(year_error_df['time_year']<year_error_df['year_min'])]

In [661]:
pred_error = year_error_df[(year_error_df['time_year']>=year_error_df['year_min']-3) & (year_error_df['time_year']<year_error_df['year_min'])]

In [663]:
pred_error[pred_error.ticker_symbol==ticker].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1338 entries, 695 to 2032
Data columns (total 8 columns):
ticker_symbol    1338 non-null object
time_year        1338 non-null int64
level_2          1338 non-null int64
organ_id         1338 non-null int64
income_err       545 non-null float64
profit_err       662 non-null float64
eps_err          1320 non-null float64
year_min         1338 non-null float64
dtypes: float64(4), int64(3), object(1)
memory usage: 94.1+ KB


In [634]:
# df = year_error_df[(year_error_df['time_year']>=year_error_df['year_min']-3)]

In [635]:
# df = df[(df['time_year']<df['year_min'])]

In [637]:
# df[df.ticker_symbol==ticker].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 426 entries, 695 to 1120
Data columns (total 8 columns):
ticker_symbol    426 non-null object
time_year        426 non-null int64
level_2          426 non-null int64
organ_id         426 non-null int64
income_err       220 non-null float64
profit_err       277 non-null float64
eps_err          420 non-null float64
year_min         426 non-null float64
dtypes: float64(4), int64(3), object(1)
memory usage: 30.0+ KB


In [664]:
year_error_df[year_error_df.ticker_symbol==ticker].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1338 entries, 695 to 2032
Data columns (total 8 columns):
ticker_symbol    1338 non-null object
time_year        1338 non-null int64
level_2          1338 non-null int64
organ_id         1338 non-null int64
income_err       545 non-null float64
profit_err       662 non-null float64
eps_err          1320 non-null float64
year_min         1338 non-null float64
dtypes: float64(4), int64(3), object(1)
memory usage: 94.1+ KB


In [652]:
# year_error_df[year_error_df.ticker_symbol==ticker]

In [665]:
pred_error.time_year.unique()

array([2016, 2017, 2018], dtype=int64)

In [666]:
pred_error.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 503122 entries, 1 to 584076
Data columns (total 8 columns):
ticker_symbol    503122 non-null object
time_year        503122 non-null int64
level_2          503122 non-null int64
organ_id         503122 non-null int64
income_err       186950 non-null float64
profit_err       279212 non-null float64
eps_err          490398 non-null float64
year_min         503122 non-null float64
dtypes: float64(4), int64(3), object(1)
memory usage: 34.5+ MB


In [75]:
pred_error.head()

,ticker_symbol,time_year,level_2,organ_id,income_err,profit_err,eps_err
1,000001,2016,0,61,NaN,0.001096,0.000906
2,000001,2016,1,9,NaN,NaN,0.000918
3,000001,2016,2,24,NaN,0.003148,0.002812
4,000001,2016,3,41,NaN,0.004058,0.003673
5,000001,2016,4,45,0.000325,0.002801,0.002812


In [81]:

def get_org_parameter(prediction, organ_err):
    """
    计算机构误差加权系数
    """
    organ_parameter = prediction.copy(deep=True).dropna(subset=['organ_id'])
    if len(organ_err) > 0:
        mean_organ_err = pd.DataFrame(
            organ_err.groupby('organ_id')[['income_err', 'profit_err', 'eps_err']].mean()).reset_index()
        income_err_mean = mean_organ_err['income_err'].mean()
        profit_err_mean = mean_organ_err['profit_err'].mean()
        eps_err_mean = mean_organ_err['eps_err'].mean()
        organ_parameter = pd.merge(mean_organ_err, organ_parameter, how='right', on='organ_id')
        organ_parameter['income_err'] = organ_parameter['income_err'].fillna(income_err_mean)
        organ_parameter['profit_err'] = organ_parameter['profit_err'].fillna(profit_err_mean)
        organ_parameter['eps_err'] = organ_parameter['eps_err'].fillna(eps_err_mean)
    else:
        organ_parameter['income_err'] = 1. / len(organ_parameter) if len(organ_parameter) > 0 else np.nan
        organ_parameter['profit_err'] = 1. / len(organ_parameter) if len(organ_parameter) > 0 else np.nan
        organ_parameter['eps_err'] = 1. / len(organ_parameter) if len(organ_parameter) > 0 else np.nan
    organ_parameter = organ_parameter.sort_values(by='create_date').reset_index().drop(labels='index', axis=1)
    organ_parameter['income_para'] = 1. / organ_parameter['income_err']
    organ_parameter['profit_para'] = 1. / organ_parameter['profit_err']
    organ_parameter['eps_para'] = 1. / organ_parameter['eps_err']
    return organ_parameter


In [114]:
pred_error.head()

,ticker_symbol,time_year,level_2,organ_id,income_err,profit_err,eps_err
1,000001,2016,0,61,NaN,0.001096,0.000906
2,000001,2016,1,9,NaN,NaN,0.000918
3,000001,2016,2,24,NaN,0.003148,0.002812
4,000001,2016,3,41,NaN,0.004058,0.003673
5,000001,2016,4,45,0.000325,0.002801,0.002812


In [174]:
pred_error.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 584102 entries, 1 to 584117
Data columns (total 7 columns):
ticker_symbol    584102 non-null object
time_year        584102 non-null int64
level_2          584102 non-null int64
organ_id         584102 non-null int64
income_err       214685 non-null float64
profit_err       330608 non-null float64
eps_err          568980 non-null float64
dtypes: float64(3), int64(3), object(1)
memory usage: 35.7+ MB


In [667]:
# group be organ id , 算出每个机构的相关系数， 再对dfmerge 的每一行做合并。   
# gropuby (organid vs ticker)
mean_org_error = pred_error[['organ_id', 'ticker_symbol', 'income_err', 'profit_err', 'eps_err']].groupby(['organ_id', 'ticker_symbol']).mean().reset_index()
# pred_org = pd.merge()
# mean_org_error

In [669]:
# pred_error[pred_error.ticker_symbol==ticker]

In [670]:
# groupby (ticker) to get mean 
mean_ticker_error = pred_error[['ticker_symbol', 'income_err', 'profit_err', 'eps_err']].groupby(['ticker_symbol']).mean().reset_index()
mean_ticker_error.rename(columns={'income_err':'t_income_err', 'profit_err':'t_profit_err', 'eps_err':'t_eps_err'}, inplace=True)

In [671]:
# mean_org_error fillna with mean_ticker_error
mean_org_error = mean_org_error.merge(mean_ticker_error, on=['ticker_symbol'], how='left')

In [673]:
# mean_org_error[mean_org_error.ticker_symbol=='000009']

In [674]:
mean_org_error.head()

,organ_id,ticker_symbol,income_err,profit_err,eps_err,t_income_err,t_profit_err,t_eps_err
0,2,000001,0.012571,0.000480,0.002360,0.014628,0.005186,0.356140
1,2,000002,0.035451,0.006243,0.007211,0.023953,0.010409,0.009097
2,2,000012,0.004183,0.774979,1.547221,0.023726,1.088865,1.809030
3,2,000026,0.002313,0.025158,0.020592,0.003333,0.016741,0.013606
4,2,000027,0.048690,0.488529,2.333523,0.032912,0.771785,1.339491


In [224]:
# to fill na 

# df.Cat1 = np.where(df.Cat1.isnull(), df.Cat2, df.Cat1)
np.where(mean_org_error.income_err.isnull(), mean_org_error.income_err, mean_org_error.t_income_err)

array([0.01462792, 0.02395283, 0.02372587, ..., 0.08008669, 0.00970308,
       0.03594058])

In [675]:
# from mean_org_error , get rows with any('income_err', 'profit_err', 'eps_err') is NA 
mean_org = mean_org_error.loc[~mean_org_error.isnull().any(axis=1)]



In [676]:
mean_org_na = mean_org_error.loc[mean_org_error.isnull().any(axis=1)]


In [677]:
# from mean_org_error , get rows with any('income_err', 'profit_err', 'eps_err') is NA 
# mean_org_na_idx = mean_org_error[(mean_org_error.income_err == ) | (mean_org_error.profit_err == np.nan) | (mean_org_error.eps_err == np.nan)].index
# mean_org_na = mean_org_error.loc[mean_org_na_idx]

In [678]:
# mean_org_na

In [742]:
mean_org_error['income_err'] = mean_org_error['income_err'].fillna(mean_org_error['t_income_err'])

KeyError: 't_income_err'

In [680]:
mean_org_error['profit_err'] = mean_org_error['profit_err'].fillna(mean_org_error['t_profit_err'])

In [681]:
mean_org_error['eps_err'] = mean_org_error['eps_err'].fillna(mean_org_error['t_eps_err'])

In [682]:
# mean_org_na['income_err'] = mean_org_na['income_err'].fillna(mean_org_na['t_income_err'])

In [213]:
# mean_org_na , use t_income_err to fill na in income_err
# mean_org_na[['income_err']]=mean_org_na[['income_err','t_income_err']].apply(lambda x: x.t_income_err, axis=1)

In [197]:
# mean_ticker_error
def fill_na(row):
#     print(row['t_income_err'])
    row[['income_err']].fillna(row['t_income_err'])
    print(row)
#     row[['profit_err']].fillna(row['t_profit_err'])
#     row[['eps_err']].fillna(row['t_eps_err'])
    return row
    
# mean_org_na.apply(fill_na, axis=1)

In [214]:
# mean_org_error = pd.concat([mean_org, mean_org_na], axis=1)

In [683]:
# mean_org_error[(mean_org_error.organ_id==2) & (mean_org_error.ticker_symbol=='000040')]

In [684]:
# to drop t_income_err	t_profit_err	t_eps_err
mean_org_error =  mean_org_error.drop(['t_income_err','t_profit_err', 't_eps_err'],axis=1)

In [ ]:
# TODO get_org_parameter
if mean_org_error is None:
    organ_parameter['income_err'] = 1. / len(organ_parameter) if len(organ_parameter) > 0 else np.nan
    organ_parameter['profit_err'] = 1. / len(organ_parameter) if len(organ_parameter) > 0 else np.nan
    organ_parameter['eps_err'] = 1. / len(organ_parameter) if len(organ_parameter) > 0 else np.nan

In [685]:
mean_org_error.head()

,organ_id,ticker_symbol,income_err,profit_err,eps_err
0,2,000001,0.012571,0.000480,0.002360
1,2,000002,0.035451,0.006243,0.007211
2,2,000012,0.004183,0.774979,1.547221
3,2,000026,0.002313,0.025158,0.020592
4,2,000027,0.048690,0.488529,2.333523


In [686]:
# TOOD get org parameter
def get_org_weight(organ_parameter):
    """
    计算机构误差加权系数
    """
#     organ_parameter = prediction.copy(deep=True).dropna(subset=['organ_id'])
#     if len(organ_err) > 0:
#         mean_organ_err = pd.DataFrame(
#             organ_err.groupby('organ_id')[['income_err', 'profit_err', 'eps_err']].mean()).reset_index()
#         income_err_mean = mean_organ_err['income_err'].mean()
#         profit_err_mean = mean_organ_err['profit_err'].mean()
#         eps_err_mean = mean_organ_err['eps_err'].mean()
#         organ_parameter = pd.merge(mean_organ_err, organ_parameter, how='right', on='organ_id')
#         organ_parameter['income_err'] = organ_parameter['income_err'].fillna(income_err_mean)
#         organ_parameter['profit_err'] = organ_parameter['profit_err'].fillna(profit_err_mean)
#         organ_parameter['eps_err'] = organ_parameter['eps_err'].fillna(eps_err_mean)
#     else:
#         organ_parameter['income_err'] = 1. / len(organ_parameter) if len(organ_parameter) > 0 else np.nan
#         organ_parameter['profit_err'] = 1. / len(organ_parameter) if len(organ_parameter) > 0 else np.nan
#         organ_parameter['eps_err'] = 1. / len(organ_parameter) if len(organ_parameter) > 0 else np.nan
    organ_parameter = organ_parameter.sort_values(by='create_date').reset_index().drop(labels='index', axis=1)
    organ_parameter['income_para'] = 1. / organ_parameter['income_err']
    organ_parameter['profit_para'] = 1. / organ_parameter['profit_err']
    organ_parameter['eps_para'] = 1. / organ_parameter['eps_err']
    return organ_parameter


In [687]:
prediction = mean_org_error[['income_err', 'profit_err', 'eps_err', 'organ_id','ticker_symbol']].merge(dfmerge, on=['organ_id','ticker_symbol'], how='right')

In [688]:
pred_org = get_org_weight(prediction)

In [241]:
# dfmerge.head()

In [689]:
pred_org.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9404 entries, 0 to 9403
Data columns (total 18 columns):
income_err          8802 non-null float64
profit_err          8802 non-null float64
eps_err             8802 non-null float64
organ_id            9404 non-null float64
ticker_symbol       9404 non-null object
report_search_id    9404 non-null int64
create_date         9404 non-null datetime64[ns]
into_date           9328 non-null object
time_year           9328 non-null float64
forecast_income     9231 non-null float64
forecast_profit     9297 non-null float64
forecast_eps        9251 non-null float64
org_name            9328 non-null object
org_remark1         9328 non-null float64
target_price        9404 non-null float64
income_para         8802 non-null float64
profit_para         8802 non-null float64
eps_para            8802 non-null float64
dtypes: datetime64[ns](1), float64(13), int64(1), object(3)
memory usage: 1.3+ MB


In [694]:
# pred_org[pred_org.ticker_symbol==ticker]

In [695]:
# dfmerge[(dfmerge.organ_id==2 ) & (dfmerge.ticker_symbol=='000012')]

In [696]:
# dfmerge[(dfmerge.ticker_symbol=='000157') ]

In [697]:
dfmerge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9404 entries, 0 to 9403
Data columns (total 12 columns):
report_search_id    9404 non-null int64
ticker_symbol       9404 non-null object
organ_id            9404 non-null float64
create_date         9404 non-null datetime64[ns]
into_date           9328 non-null object
time_year           9328 non-null float64
forecast_income     9231 non-null float64
forecast_profit     9297 non-null float64
forecast_eps        9251 non-null float64
org_name            9328 non-null object
org_remark1         9328 non-null float64
target_price        9404 non-null float64
dtypes: datetime64[ns](1), float64(7), int64(1), object(3)
memory usage: 955.1+ KB


In [93]:
mean_org_error.head()

,organ_id,ticker_symbol,income_err,profit_err,eps_err
0,2,000001,0.012571,0.000480,0.002360
1,2,000002,0.035451,0.006243,0.007211
2,2,000012,0.004183,0.774979,1.547221
3,2,000018,2.744803,1.148425,1.411762
4,2,000026,0.002313,0.025158,0.020592


In [242]:
# pred_org = get_org_parameter(dfmerge, pred_error)

In [243]:
# pred_org

In [779]:
pred_org.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9404 entries, 0 to 9403
Data columns (total 18 columns):
income_err          8802 non-null float64
profit_err          8802 non-null float64
eps_err             8802 non-null float64
organ_id            9404 non-null float64
ticker_symbol       9404 non-null object
report_search_id    9404 non-null int64
create_date         9404 non-null datetime64[ns]
into_date           9328 non-null object
time_year           9328 non-null float64
forecast_income     9231 non-null float64
forecast_profit     9297 non-null float64
forecast_eps        9251 non-null float64
org_name            9328 non-null object
org_remark1         9328 non-null float64
target_price        9404 non-null float64
income_para         8802 non-null float64
profit_para         8802 non-null float64
eps_para            8802 non-null float64
dtypes: datetime64[ns](1), float64(13), int64(1), object(3)
memory usage: 1.3+ MB


In [852]:

# if len(pred_org) > 0:
para_name = 'eps'
# pred_org_by_year = pred_org.sort_values(['report_search_id', 'time_year']).drop_duplicates('report_search_id', keep='first')
# translate to ->
pred_org_by_year = pred_org.sort_values(['report_search_id', 'time_year']).groupby('ticker_symbol').apply(lambda x: x.drop_duplicates('report_search_id', keep='first'))#.drop_index()#.reset_index()

In [853]:
pred_org_by_year.index=pred_org_by_year.index.droplevel(0)#.reset_index()

In [854]:
pred_org_by_year = pred_org_by_year.reset_index()

In [855]:
pred_org_by_year.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3266 entries, 0 to 3265
Data columns (total 19 columns):
index               3266 non-null int64
income_err          3015 non-null float64
profit_err          3015 non-null float64
eps_err             3015 non-null float64
organ_id            3266 non-null float64
ticker_symbol       3266 non-null object
report_search_id    3266 non-null int64
create_date         3266 non-null datetime64[ns]
into_date           3190 non-null object
time_year           3190 non-null float64
forecast_income     3154 non-null float64
forecast_profit     3178 non-null float64
forecast_eps        3161 non-null float64
org_name            3190 non-null object
org_remark1         3190 non-null float64
target_price        3266 non-null float64
income_para         3015 non-null float64
profit_para         3015 non-null float64
eps_para            3015 non-null float64
dtypes: datetime64[ns](1), float64(13), int64(2), object(3)
memory usage: 484.9+ KB


In [856]:
pred_org_by_year.head()

,index,income_err,profit_err,eps_err,organ_id,ticker_symbol,report_search_id,create_date,into_date,time_year,forecast_income,forecast_profit,forecast_eps,org_name,org_remark1,target_price,income_para,profit_para,eps_para
0,1181,0.006476,0.000775,0.001209,77.0,000001,2099096,2019-03-08,2019-03-08 00:00:00,2019.0,1.295250e+11,2.741200e+10,1.60,华泰证券,1.0,15.70,154.427048,1289.831245,827.196836
1,1945,0.013207,0.002713,0.005742,3.0,000001,2101297,2019-03-17,2019-03-17 00:00:00,2019.0,1.301910e+11,2.916500e+10,1.70,中金,1.0,16.52,75.717121,368.549425,174.149401
2,9291,0.019298,0.017020,0.007689,2570.0,000001,2110384,2019-04-24,2019-04-24 00:00:00,2019.0,1.335000e+11,2.900000e+10,1.69,天风证券,1.0,17.21,51.818206,58.755249,130.061402
3,9166,0.012832,0.016905,0.017603,20.0,000001,2110425,2019-04-24,2019-04-24 00:00:00,2018.0,1.167160e+11,2.481800e+10,1.39,中信建投,1.0,17.00,77.928826,59.152505,56.808885
4,9152,0.000738,0.000060,0.002536,30.0,000001,2110528,2019-04-24,2019-04-24 00:00:00,2019.0,1.399790e+11,2.790000e+10,1.62,西南证券,1.0,17.33,1354.778534,16535.561331,394.305979


In [857]:
# inf_index = np.where(pred_org_by_year[para_name + '_para'] == np.inf)[0]
pred_org.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9404 entries, 0 to 9403
Data columns (total 18 columns):
income_err          8802 non-null float64
profit_err          8802 non-null float64
eps_err             8802 non-null float64
organ_id            9404 non-null float64
ticker_symbol       9404 non-null object
report_search_id    9404 non-null int64
create_date         9404 non-null datetime64[ns]
into_date           9328 non-null object
time_year           9328 non-null float64
forecast_income     9231 non-null float64
forecast_profit     9297 non-null float64
forecast_eps        9251 non-null float64
org_name            9328 non-null object
org_remark1         9328 non-null float64
target_price        9404 non-null float64
income_para         8802 non-null float64
profit_para         8802 non-null float64
eps_para            8802 non-null float64
dtypes: datetime64[ns](1), float64(13), int64(1), object(3)
memory usage: 1.3+ MB


In [858]:
pred_org[pred_org.eps_err.isnull()].ticker_symbol.unique()

array(['002463', '000708', '300487', '002027', '002675', '300347',
       '603127', '002364', '600036', '603806', '002572', '300059',
       '300168', '603801', '600231', '000661', '600702', '300638',
       '002056', '300657', '002716', '603181', '600158', '300617',
       '603136', '600027', '603456', '002916', '002233', '603659',
       '300741', '002294', '600050', '002807', '603960', '000860',
       '300590', '300409', '603881', '000600', '002821', '601066',
       '300137', '000768', '002436', '603379', '000786', '603688',
       '300446', '600011', '300017', '601211', '002100', '002468',
       '603186', '600030', '603214', '002035', '603259', '601319',
       '002734', '601601', '601233', '601339', '002318', '002013',
       '600206', '002041', '603969', '603686', '603357', '603899',
       '002408', '603055', '300296', '000063', '300045', '002875',
       '300357', '300233', '601699', '603711', '601828', '601628',
       '002672', '600837', '300047', '002396', '300570', '6018

In [859]:
pred_org_by_year[pred_org_by_year.ticker_symbol==ticker]

,index,income_err,profit_err,eps_err,organ_id,ticker_symbol,report_search_id,create_date,into_date,time_year,forecast_income,forecast_profit,forecast_eps,org_name,org_remark1,target_price,income_para,profit_para,eps_para
7,3527,0.013270,0.006357,0.006399,7104.0,000002,2102230,2019-03-26,2019-03-26 00:00:00,2019.0,4.193150e+11,4.043000e+10,3.66,华创证券,1.0,45.00,75.355967,157.305364,156.285570
8,3371,0.024497,0.003101,0.003172,30.0,000002,2102345,2019-03-26,2019-03-27 00:00:00,2019.0,3.607988e+11,4.212510e+10,3.82,西南证券,1.0,34.38,40.821131,322.470080,315.280689
9,3780,0.027357,0.004129,0.033487,119.0,000002,2102347,2019-03-27,2019-03-27 00:00:00,2019.0,3.676340e+11,4.266400e+10,3.86,东北证券,1.0,41.13,36.553823,242.181483,29.862420
10,3643,0.120886,0.008121,0.009399,12.0,000002,2102450,2019-03-26,2019-03-27 00:00:00,2019.0,3.642920e+11,4.072400e+10,3.69,东方证券,1.0,33.21,8.272242,123.132865,106.389315
11,3604,0.002336,0.002027,0.002093,77.0,000002,2102915,2019-03-26,2019-03-28 00:00:00,2019.0,3.593380e+11,3.994500e+10,3.62,华泰证券,1.0,32.58,428.131187,493.451103,477.734516
12,3440,0.025932,0.013952,0.010488,13.0,000002,2103228,2019-03-26,2019-03-26 00:00:00,2019.0,3.644736e+11,4.090360e+10,3.71,中信证券,1.0,37.05,38.563108,71.672605,95.343302
13,5763,0.009821,0.013091,0.012579,20.0,000002,2104598,2019-04-04,2019-04-04 00:00:00,2019.0,3.451000e+11,4.011600e+10,3.63,中信建投,1.0,35.00,101.827050,76.387846,79.500354
14,6030,0.009436,0.006292,0.006323,61.0,000002,2105829,2019-04-08,2019-04-08 00:00:00,2019.0,3.579160e+11,3.933100e+10,3.56,广发证券,1.0,39.60,105.972046,158.925449,158.141320
15,7277,0.025782,0.003821,0.003874,5.0,000002,2108070,2019-04-16,2019-04-16 00:00:00,2019.0,3.584900e+11,4.101100e+10,3.63,光大证券,1.0,39.91,38.787347,261.694321,258.141122
16,7444,0.035451,0.006243,0.007211,2.0,000002,2108407,2019-04-16,2019-04-17 00:00:00,2019.0,3.572150e+11,4.313300e+10,3.82,国泰君安,1.0,43.01,28.208108,160.169956,138.678339


In [860]:
df = pred_org[pred_org.ticker_symbol=='002463']

In [861]:
# df

In [757]:
df = df.iloc[[0]]

In [758]:
np.isnan(df['eps' + '_err'].values).all() # 当所有行均为na时返回True, 否则返回False

True

In [708]:
np.isnan(pred_org['eps' + '_err'].values).all()

False

In [739]:
# pred_org_by_year[pred_org_by_year.eps_err.isnull()]

In [740]:
# TODO fillna for eps_err
#                 if len(pred_org_by_year) > 0 and np.isnan(pred_org[para_name + '_err'].values).all():


In [862]:
nan_eps_err = pred_org_by_year.groupby('ticker_symbol').apply(lambda x: np.isnan(x['eps' + '_err'].values).all()).to_frame('nan_eps_err').reset_index()

In [863]:
nan_eps_err[nan_eps_err.ticker_symbol==ticker]

,ticker_symbol,nan_eps_err
1,000002,False


In [864]:
pred_org_by_year=pred_org_by_year.merge(nan_eps_err,on='ticker_symbol') 

In [865]:
year_count_df = pred_org_by_year.groupby('ticker_symbol').size().to_frame('group_count').reset_index()

In [866]:
# year_count_df

In [867]:
pred_org_by_year=pred_org_by_year.merge(year_count_df,on='ticker_symbol') 

In [871]:
pred_org_by_year.head()

,index,income_err,profit_err,eps_err,organ_id,ticker_symbol,report_search_id,create_date,into_date,time_year,...,forecast_profit,forecast_eps,org_name,org_remark1,target_price,income_para,profit_para,eps_para,nan_eps_err,group_count
0,1181,0.006476,0.000775,0.001209,77.0,000001,2099096,2019-03-08,2019-03-08 00:00:00,2019.0,...,2.741200e+10,1.60,华泰证券,1.0,15.70,154.427048,1289.831245,827.196836,False,7
1,1945,0.013207,0.002713,0.005742,3.0,000001,2101297,2019-03-17,2019-03-17 00:00:00,2019.0,...,2.916500e+10,1.70,中金,1.0,16.52,75.717121,368.549425,174.149401,False,7
2,9291,0.019298,0.017020,0.007689,2570.0,000001,2110384,2019-04-24,2019-04-24 00:00:00,2019.0,...,2.900000e+10,1.69,天风证券,1.0,17.21,51.818206,58.755249,130.061402,False,7
3,9166,0.012832,0.016905,0.017603,20.0,000001,2110425,2019-04-24,2019-04-24 00:00:00,2018.0,...,2.481800e+10,1.39,中信建投,1.0,17.00,77.928826,59.152505,56.808885,False,7
4,9152,0.000738,0.000060,0.002536,30.0,000001,2110528,2019-04-24,2019-04-24 00:00:00,2019.0,...,2.790000e+10,1.62,西南证券,1.0,17.33,1354.778534,16535.561331,394.305979,False,7


In [869]:
pred_org_by_year.loc[:,'eps_err']= pred_org_by_year.apply(lambda x: 1. / x['group_count'] if x['nan_eps_err'] is True and x['group_count']>0 else x['eps_err'], axis=1)

In [870]:
pred_org_by_year.loc[:,'eps_para']= pred_org_by_year.apply(lambda x: 1. / x['eps_err'] if x['nan_eps_err'] is True and x['group_count']>0 else x['eps_para'], axis=1)

In [743]:
# pred_org_by_year.eps_err.isnull()
# pred_org_by_year['eps_err'] = pred_org_by_year['eps_err'].fillna(1. / pred_org_by_year['group_count'])


In [807]:
# pred_org_by_year[para_name + '_para'] = 1. / pred_org_by_year[para_name + '_err']

In [744]:
# pred_org_by_year.loc[:,'eps_err']= 1. / pred_org_by_year['group_count'] if pred_org_by_year['nan_eps_err'] else pred_org_by_year['eps_err']

In [838]:
# if len(pred_org_by_year) > 0 and np.isnan(pred_org['eps' + '_err'].values).all():
# TODO fillna
# pred_org_by_year[para_name + '_err'] = 1. / len(pred_org_by_year)
# pred_org_by_year[para_name + '_para'] = 1. / pred_org_by_year[para_name + '_err']

In [839]:
# pred_org_by_year = pred_org_by_year.reset_index().drop('index', axis=1)

In [888]:
pred_org_by_year.head()

,index,income_err,profit_err,eps_err,organ_id,ticker_symbol,report_search_id,create_date,into_date,time_year,...,forecast_profit,forecast_eps,org_name,org_remark1,target_price,income_para,profit_para,eps_para,nan_eps_err,group_count
0,1181,0.006476,0.000775,0.001209,77.0,000001,2099096,2019-03-08,2019-03-08 00:00:00,2019.0,...,2.741200e+10,1.60,华泰证券,1.0,15.70,154.427048,1289.831245,827.196836,False,7
1,1945,0.013207,0.002713,0.005742,3.0,000001,2101297,2019-03-17,2019-03-17 00:00:00,2019.0,...,2.916500e+10,1.70,中金,1.0,16.52,75.717121,368.549425,174.149401,False,7
2,9291,0.019298,0.017020,0.007689,2570.0,000001,2110384,2019-04-24,2019-04-24 00:00:00,2019.0,...,2.900000e+10,1.69,天风证券,1.0,17.21,51.818206,58.755249,130.061402,False,7
3,9166,0.012832,0.016905,0.017603,20.0,000001,2110425,2019-04-24,2019-04-24 00:00:00,2018.0,...,2.481800e+10,1.39,中信建投,1.0,17.00,77.928826,59.152505,56.808885,False,7
4,9152,0.000738,0.000060,0.002536,30.0,000001,2110528,2019-04-24,2019-04-24 00:00:00,2019.0,...,2.790000e+10,1.62,西南证券,1.0,17.33,1354.778534,16535.561331,394.305979,False,7


In [873]:
# 
# inf_index = np.where(pred_org_by_year[para_name + '_para'] == np.inf)[0]
inf_err = pred_org_by_year.groupby('ticker_symbol').apply(lambda x: len(np.where(x[para_name + '_para'] == np.inf)[0])).to_frame('inf_err').reset_index()

In [874]:
inf_err.head()

,ticker_symbol,inf_err
0,000001,0
1,000002,0
2,000012,0
3,000026,0
4,000027,0


In [935]:
# inf_err
prediction=pred_org_by_year.merge(inf_err, on='ticker_symbol')

In [936]:
prediction.head()

,level_0,income_err,profit_err,eps_err,organ_id,ticker_symbol,report_search_id,create_date,into_date,time_year,...,org_remark1,target_price,income_para,profit_para,eps_para,nan_eps_err,group_count,decay_day,decay_day_eps,inf_err
0,0,0.006476,0.000775,0.001209,77.0,000001,2099096,2019-03-08,2019-03-08 00:00:00,2019.0,...,1.0,15.70,154.427048,1289.831245,827.196836,False,7,0.64,529.405975,0
1,1,0.013207,0.002713,0.005742,3.0,000001,2101297,2019-03-17,2019-03-17 00:00:00,2019.0,...,1.0,16.52,75.717121,368.549425,174.149401,False,7,0.64,111.455616,0
2,2,0.019298,0.017020,0.007689,2570.0,000001,2110384,2019-04-24,2019-04-24 00:00:00,2019.0,...,1.0,17.21,51.818206,58.755249,130.061402,False,7,0.80,104.049121,0
3,3,0.012832,0.016905,0.017603,20.0,000001,2110425,2019-04-24,2019-04-24 00:00:00,2018.0,...,1.0,17.00,77.928826,59.152505,56.808885,False,7,0.80,45.447108,0
4,4,0.000738,0.000060,0.002536,30.0,000001,2110528,2019-04-24,2019-04-24 00:00:00,2019.0,...,1.0,17.33,1354.778534,16535.561331,394.305979,False,7,0.80,315.444783,0


In [937]:
# prediction.apply(lambda x: 1./x['inf_err'] if x['eps_para']==np.inf and x['inf_err']>0 else x['eps_para'], axis=1).reset_index()

In [938]:
prediction.loc[:,'eps_para']=prediction.apply(lambda x: 1./x['inf_err'] if x['eps_para']==np.inf and x['inf_err']>0 else x['eps_para'], axis=1)#.reset_index()

In [939]:
prediction.head()

,level_0,income_err,profit_err,eps_err,organ_id,ticker_symbol,report_search_id,create_date,into_date,time_year,...,org_remark1,target_price,income_para,profit_para,eps_para,nan_eps_err,group_count,decay_day,decay_day_eps,inf_err
0,0,0.006476,0.000775,0.001209,77.0,000001,2099096,2019-03-08,2019-03-08 00:00:00,2019.0,...,1.0,15.70,154.427048,1289.831245,827.196836,False,7,0.64,529.405975,0
1,1,0.013207,0.002713,0.005742,3.0,000001,2101297,2019-03-17,2019-03-17 00:00:00,2019.0,...,1.0,16.52,75.717121,368.549425,174.149401,False,7,0.64,111.455616,0
2,2,0.019298,0.017020,0.007689,2570.0,000001,2110384,2019-04-24,2019-04-24 00:00:00,2019.0,...,1.0,17.21,51.818206,58.755249,130.061402,False,7,0.80,104.049121,0
3,3,0.012832,0.016905,0.017603,20.0,000001,2110425,2019-04-24,2019-04-24 00:00:00,2018.0,...,1.0,17.00,77.928826,59.152505,56.808885,False,7,0.80,45.447108,0
4,4,0.000738,0.000060,0.002536,30.0,000001,2110528,2019-04-24,2019-04-24 00:00:00,2019.0,...,1.0,17.33,1354.778534,16535.561331,394.305979,False,7,0.80,315.444783,0


In [252]:

# if len(inf_index) != 0:
#     pred_org_by_year[para_name + '_para'] = pred_org_by_year[para_name + '_para'].apply(
#         lambda x: 1. / len(inf_index) if x == np.inf else 0.)

In [940]:

prediction = prediction.dropna(subset=[para_name + '_para'])

In [941]:
pred_org_by_year.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3266 entries, 0 to 3265
Data columns (total 23 columns):
level_0             3266 non-null int64
income_err          3015 non-null float64
profit_err          3015 non-null float64
eps_err             3074 non-null float64
organ_id            3266 non-null float64
ticker_symbol       3266 non-null object
report_search_id    3266 non-null int64
create_date         3266 non-null datetime64[ns]
into_date           3190 non-null object
time_year           3190 non-null float64
forecast_income     3154 non-null float64
forecast_profit     3178 non-null float64
forecast_eps        3161 non-null float64
org_name            3190 non-null object
org_remark1         3190 non-null float64
target_price        3266 non-null float64
income_para         3015 non-null float64
profit_para         3015 non-null float64
eps_para            3074 non-null float64
nan_eps_err         3266 non-null bool
group_count         3266 non-null int64
decay_day       

In [942]:



# if len(pred_org_by_year) > 0:
prediction['decay_day'] = prediction['create_date'].apply(
    lambda x: 0.8 ** int((datetime.strptime(str(pred_date), '%Y-%m-%d') - x).days / 30))
prediction['decay_day_' + para_name] = prediction['decay_day'] * pred_org_by_year[
    para_name + '_para']
prediction = prediction.reset_index().drop('index', axis=1)


In [896]:
# pred_org_by_year

In [898]:
# nan_index = np.where(np.isnan(pred_org_by_year['target_price']))[0]

In [901]:
# pred_org_by_year[pred_org_by_year.ticker_symbol=='000157']

In [943]:
# nan_sum = sum(pred_org_by_year['decay_day_' + para_name].iloc[nan_index])
# nan_index = np.where(np.isnan(pred_org_by_year[self.consensus_name]))[0]
#                     nan_sum = sum(pred_org_by_year['decay_day_' + para_name].iloc[nan_index])
                    
nan_sum_df = prediction.groupby('ticker_symbol')['target_price'].apply(lambda x:x.isnull().sum()).to_frame().reset_index()

In [300]:
# nan_sum_df.reset_index()

In [904]:
# nan_sum = sum(pred_org_by_year['decay_day_' + para_name].iloc[nan_index])
# prediction.ticker_symbol[prediction.target_price.isnull()].value_counts()

In [906]:
# nan_sum = sum(pred_org_by_year['decay_day_' + para_name].iloc[nan_index])
# prediction['target_price'].isnull().groupby(pred_org_by_year['ticker_symbol']).sum()

In [301]:
# nan_sum = sum(pred_org_by_year['decay_day_' + para_name].iloc[nan_index])
# pred_org_by_year['target_price'].isnull().groupby(pred_org_by_year['ticker_symbol']).count()

In [944]:
nan_sum_df.rename(columns={'target_price':'target_price_nan_sum'}, inplace=True)

In [945]:
nan_sum_df.head()

,ticker_symbol,target_price_nan_sum
0,000001,0
1,000002,0
2,000012,0
3,000026,0
4,000027,0


In [930]:
# pred_org_by_year

In [946]:
prediction = prediction.merge(nan_sum_df, on='ticker_symbol', how='left')

In [947]:
# prediction['decay_day_eps'].sum()

In [933]:
# prediction[['ticker_symbol', 'target_price','decay_day_' + para_name,'target_price_nan_sum']].head()

In [948]:
prediction.head()

,level_0,income_err,profit_err,eps_err,organ_id,ticker_symbol,report_search_id,create_date,into_date,time_year,...,target_price,income_para,profit_para,eps_para,nan_eps_err,group_count,decay_day,decay_day_eps,inf_err,target_price_nan_sum
0,0,0.006476,0.000775,0.001209,77.0,000001,2099096,2019-03-08,2019-03-08 00:00:00,2019.0,...,15.70,154.427048,1289.831245,827.196836,False,7,0.64,529.405975,0,0
1,1,0.013207,0.002713,0.005742,3.0,000001,2101297,2019-03-17,2019-03-17 00:00:00,2019.0,...,16.52,75.717121,368.549425,174.149401,False,7,0.64,111.455616,0,0
2,2,0.019298,0.017020,0.007689,2570.0,000001,2110384,2019-04-24,2019-04-24 00:00:00,2019.0,...,17.21,51.818206,58.755249,130.061402,False,7,0.80,104.049121,0,0
3,3,0.012832,0.016905,0.017603,20.0,000001,2110425,2019-04-24,2019-04-24 00:00:00,2018.0,...,17.00,77.928826,59.152505,56.808885,False,7,0.80,45.447108,0,0
4,4,0.000738,0.000060,0.002536,30.0,000001,2110528,2019-04-24,2019-04-24 00:00:00,2019.0,...,17.33,1354.778534,16535.561331,394.305979,False,7,0.80,315.444783,0,0


In [911]:
# pred_org_by_year[['ticker_symbol','decay_day_' + para_name]].groupby('ticker_symbol').sum()

In [949]:
# pred_org_by_year[['ticker_symbol','target_price_nan_sum']]
decay_sum = prediction[['ticker_symbol','decay_day_' + para_name]].groupby('ticker_symbol').sum().reset_index()

In [950]:
decay_sum.rename(columns={'decay_day_eps':'decay_sum'},inplace=True)

In [951]:
prediction = prediction.merge(decay_sum, on='ticker_symbol')

In [954]:
prediction.head()

,level_0,income_err,profit_err,eps_err,organ_id,ticker_symbol,report_search_id,create_date,into_date,time_year,...,profit_para,eps_para,nan_eps_err,group_count,decay_day,decay_day_eps,inf_err,target_price_nan_sum,decay_sum,expec
0,0,0.006476,0.000775,0.001209,77.0,000001,2099096,2019-03-08,2019-03-08 00:00:00,2019.0,...,1289.831245,827.196836,False,7,0.64,529.405975,0,0,1672.895978,4.968434
1,1,0.013207,0.002713,0.005742,3.0,000001,2101297,2019-03-17,2019-03-17 00:00:00,2019.0,...,368.549425,174.149401,False,7,0.64,111.455616,0,0,1672.895978,1.100634
2,2,0.019298,0.017020,0.007689,2570.0,000001,2110384,2019-04-24,2019-04-24 00:00:00,2019.0,...,58.755249,130.061402,False,7,0.80,104.049121,0,0,1672.895978,1.070410
3,3,0.012832,0.016905,0.017603,20.0,000001,2110425,2019-04-24,2019-04-24 00:00:00,2018.0,...,59.152505,56.808885,False,7,0.80,45.447108,0,0,1672.895978,0.461834
4,4,0.000738,0.000060,0.002536,30.0,000001,2110528,2019-04-24,2019-04-24 00:00:00,2019.0,...,16535.561331,394.305979,False,7,0.80,315.444783,0,0,1672.895978,3.267781


In [959]:

#                     pred_org_by_year['decay_' + para_name] = pred_org_by_year[self.consensus_name] * (
#                             pred_org_by_year['decay_day_' + para_name] / (
#                             pred_org_by_year['decay_day_' + para_name].sum() - nan_sum))

prediction.loc[:,'decay_eps'] = prediction['target_price'] * (prediction['decay_day_' + para_name] / (prediction['decay_sum'] - prediction['target_price_nan_sum']))

In [359]:
# TODO why apply is not ok
# pred_org_by_year.loc[:,'expec'] = pred_org_by_year.apply(
#     lambda x: x['target_price'] * (x['decay_day_' + para_name] / (x['decay_sum'] - x['target_price_nan_sum'])))

In [347]:
# con_expec = pred_org_by_year[['ticker_symbol','target_price','decay_day_' + para_name,'target_price_nan_sum','decay_sum']].groupby('ticker_symbol').apply(
#     lambda x: x['target_price'] * (x['decay_day_' + para_name] / (
#     x['decay_sum'] - x['target_price_nan_sum']))).reset_index()

In [386]:
# con_expec

In [370]:
# pred_org_by_year.groupby('ticker_symbol').size()#.reset_index()

In [976]:
def type_map(row):
    if row['con_type_target_price'] >= 3:
        return 1
    else:
        return 2
con_expec = prediction.groupby('ticker_symbol').size().to_frame('con_type_target_price').reset_index()
con_expec['con_type_target_price'] = con_expec.apply(type_map, axis=1)


In [962]:
prediction.head()

,level_0,income_err,profit_err,eps_err,organ_id,ticker_symbol,report_search_id,create_date,into_date,time_year,...,eps_para,nan_eps_err,group_count,decay_day,decay_day_eps,inf_err,target_price_nan_sum,decay_sum,expec,decay_eps
0,0,0.006476,0.000775,0.001209,77.0,000001,2099096,2019-03-08,2019-03-08 00:00:00,2019.0,...,827.196836,False,7,0.64,529.405975,0,0,1672.895978,4.968434,4.968434
1,1,0.013207,0.002713,0.005742,3.0,000001,2101297,2019-03-17,2019-03-17 00:00:00,2019.0,...,174.149401,False,7,0.64,111.455616,0,0,1672.895978,1.100634,1.100634
2,2,0.019298,0.017020,0.007689,2570.0,000001,2110384,2019-04-24,2019-04-24 00:00:00,2019.0,...,130.061402,False,7,0.80,104.049121,0,0,1672.895978,1.070410,1.070410
3,3,0.012832,0.016905,0.017603,20.0,000001,2110425,2019-04-24,2019-04-24 00:00:00,2018.0,...,56.808885,False,7,0.80,45.447108,0,0,1672.895978,0.461834,0.461834
4,4,0.000738,0.000060,0.002536,30.0,000001,2110528,2019-04-24,2019-04-24 00:00:00,2019.0,...,394.305979,False,7,0.80,315.444783,0,0,1672.895978,3.267781,3.267781


In [968]:
con_expec.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1121 entries, 0 to 1120
Data columns (total 4 columns):
ticker_symbol            1121 non-null object
con_type_target_price    1121 non-null int64
pred_date                1121 non-null object
con_target_price         0 non-null float64
dtypes: float64(1), int64(1), object(2)
memory usage: 35.1+ KB


In [977]:
con_expec.loc[:,'pred_date'] = pred_date

In [974]:
# con_expec.loc[:, 'con_target_price'] = prediction[['ticker_symbol','decay_eps']].groupby('ticker_symbol').apply(
#     lambda x: x['decay_eps'].sum() if not np.isnan(x['decay_eps']).all() else np.nan)
#.reset_index()
# con_expec['con_type_target_price'] = con_expec.apply(type_map, axis=1)


In [978]:
df = prediction[['ticker_symbol','decay_eps']].groupby('ticker_symbol').apply(
    lambda x: x['decay_eps'].sum() if not np.isnan(x['decay_eps']).all() else np.nan).to_frame("con_target_price").reset_index()
# con_expec['con_type_target_price'] = con_expec.apply(type_map, axis=1)


In [965]:
# con_expec.loc[:, 'code'] = ticker
# # con_expec.ix[0, 'pred_date'] = pred_date
# con_expec.ix[0, 'con_' + 'target_price'] = pred_org_by_year[
#     'decay_' + para_name].sum() if not np.isnan(
#     pred_org_by_year['decay_' + para_name]).all() else np.nan

In [980]:
con_expec = con_expec.merge(df, on='ticker_symbol')

In [981]:
con_expec[con_expec.ticker_symbol==ticker]

,ticker_symbol,con_type_target_price,pred_date,con_target_price
1,000002,1,2019-05-27,37.094484


In [ ]:
# for other tickers with target price is nan

In [983]:
tickers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3750 entries, 0 to 3749
Data columns (total 4 columns):
ticker_symbol     3750 non-null object
list_date         3727 non-null object
delist_date       103 non-null object
SEC_SHORT_NAME    3750 non-null object
dtypes: object(4)
memory usage: 146.5+ KB


In [984]:
tickers.head()

,ticker_symbol,list_date,delist_date,SEC_SHORT_NAME
0,000001,1991-04-03,NaN,平安银行
1,000002,1991-01-29,NaN,万科A
2,000003,1991-07-03,2002-06-14,PT金田A
3,000004,1991-01-14,NaN,国农科技
4,000005,1990-12-10,NaN,世纪星源


In [991]:
alive_ticker_df = tickers.loc[tickers.delist_date.isnull()]

In [993]:
ticker_pred_price = alive_ticker_df[['ticker_symbol']].merge(con_expec,on='ticker_symbol',how='left') 

In [997]:
ticker_pred_price['con_type_target_price'] = ticker_pred_price['con_type_target_price'].fillna(3)

In [998]:
ticker_pred_price['pred_date'] = ticker_pred_price['pred_date'].fillna(pred_date)

In [1001]:
ticker_pred_price['con_type_target_price']=ticker_pred_price['con_type_target_price'].astype(int)

In [999]:
ticker_pred_price.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3647 entries, 0 to 3646
Data columns (total 4 columns):
ticker_symbol            3647 non-null object
con_type_target_price    3647 non-null float64
pred_date                3647 non-null object
con_target_price         1109 non-null float64
dtypes: float64(2), object(2)
memory usage: 142.5+ KB


In [1002]:
ticker_pred_price

,ticker_symbol,con_type_target_price,pred_date,con_target_price
0,000001,1,2019-05-27,17.610732
1,000002,1,2019-05-27,37.094484
2,000004,3,2019-05-27,NaN
3,000005,3,2019-05-27,NaN
4,000006,3,2019-05-27,NaN
5,000007,3,2019-05-27,NaN
6,000008,3,2019-05-27,NaN
7,000009,3,2019-05-27,NaN
8,000010,3,2019-05-27,NaN
9,000011,3,2019-05-27,NaN


# done calculate price 